In [1]:
!pip install gradio
!pip install langchain_google_genai
!pip install tiktoken openai langchain --quiet
!pip install langchain-community langchain-google-vertexai beautifulsoup4 langgraph pypdf gradio --quiet
!pip install matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9

In [2]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langgraph.constants import Send
from langgraph.graph import END, START, StateGraph
from langchain.document_loaders import PyPDFLoader
import textwrap
from typing import List, Literal, TypedDict

DEVELOPING GRADIO FOR  LLM ADVANCED MODELS

In [3]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=3ce4e0c428adc995af0beeb3d076a31508924490d026b52cd84634dd6940ca5e
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
  Attempting uninstall: aiofiles
    Found existing installation: aiofiles 23.2.1
    Uninstalling aiofile

In [4]:
!pip install gradio langchain langchain-google-genai langchain-community pypdf langgraph --quiet

import os
from langchain.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain.chains.summarize import load_summarize_chain
from langchain_google_genai import ChatGoogleGenerativeAI
import gradio as gr



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.28.0 requires aiofiles>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.


In [5]:

os.environ["GOOGLE_API_KEY"] = "AIzaSyCZCb***"

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)




In [6]:

def summarize_pdf(pdf_file, method):
    loader = PyPDFLoader(pdf_file.name)
    docs = loader.load_and_split()
    chain = load_summarize_chain(llm, chain_type=method)
    summary = chain.invoke(docs)
    return summary['output_text']



In [7]:

def summarize_url(url, method):
    from langchain.document_loaders import UnstructuredURLLoader
    loader = UnstructuredURLLoader(urls=[url])
    docs = loader.load()
    chain = load_summarize_chain(llm, chain_type=method)
    summary = chain.invoke(docs)
    return summary['output_text']



In [8]:

def summarize_text(input_text, method):
    docs = [Document(page_content=input_text)]
    chain = load_summarize_chain(llm, chain_type=method)
    summary = chain.invoke(docs)
    return summary['output_text']



In [9]:

def summarize(content_type, method, pdf=None, url=None, text=None):
    try:
        if content_type == "PDF File" and pdf is not None:
            return summarize_pdf(pdf, method)
        elif content_type == "URL" and url is not None:
            return summarize_url(url, method)
        elif content_type == "Text Input" and text is not None:
            return summarize_text(text, method)
        else:
            return "Please provide valid input for the selected content type."
    except Exception as e:
        return f"Error: {str(e)}"



In [10]:

with gr.Blocks() as interface:
    gr.Markdown("# Summarization Tool with Method Selection")
    gr.Markdown(
        "Upload a **PDF**, provide a **URL**, or enter **text** to generate a summary. "
        "Choose between the **Refine** and **MapReduce** summarization methods."
    )

    content_type = gr.Radio(
        choices=["PDF File", "URL", "Text Input"],
        label="Select Content Type",
        value="PDF File",
    )

    method_type = gr.Radio(
        choices=["refine", "map_reduce"],
        label="Select Summarization Method",
        value="refine",
    )

    pdf_input = gr.File(label="Upload PDF File", file_types=[".pdf"], visible=True)
    url_input = gr.Textbox(label="Enter URL", visible=False)
    text_input = gr.Textbox(label="Enter Text", visible=False)

    result = gr.Textbox(label="Summary Output", interactive=False)


    def update_inputs(content_type):
        if content_type == "PDF File":
            return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)
        elif content_type == "URL":
            return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)
        elif content_type == "Text Input":
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)

    content_type.change(
        update_inputs,
        inputs=[content_type],
        outputs=[pdf_input, url_input, text_input]
    )


    summarize_button = gr.Button("Summarize")
    summarize_button.click(
        summarize,
        inputs=[content_type, method_type, pdf_input, url_input, text_input],
        outputs=result,
    )



In [11]:

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ba8b39a9886bc014b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
